## Science Pipelines Parquet Column Review 

### Team Meeting Sept 15 2021
(Timing before DP0.2 processing not coincidence)

#### Background

The [DPDD](https://lse-163.lsst.io/) was written a long time ago, and was not meant to be prescriptive in terms of column names. At some times it is overprescriptive in its assumptions of algorithms baked into the column names. The [baseline schema](https://github.com/lsst/sdm_schemas/blob/master/yml/baselineSchema.yaml) was also written a long time ago, and contains many tables we haven't talked about. Both of these provide guidance, but are not requirements.


This meeting is to review the existing parquet tables being *output* from the pipelines during a DRP. We've been slowly adding these outputs and adding to them over the years. Its time to look at them as a collective and reconcile inconsistencies. Let's record questions and items and to-do items for w_2021_40. The next iteration of this will be presented at the SST for feedback from the other DM product owners. 

#### Assumptions

1) If we need a column for QA, then a scientist will also need it to characterize the dataset to understand their results. We must eat our dog food. Therefore, we've errored on the side of overinclusion. 

---

##  The Source Table


* **dimensions:** visit
* **specification:** https://github.com/lsst/obs_lsst/blob/master/policy/imsim/Source.yaml
* **what**: No jointcal/FGCM for DC2 because it has a perfect reference catalog. Therefore single-frame processing is not iterative and you can assume that this is the FINAL Source Table. (This is not true for HSC, where our SourceTable is the preliminary one used for input into glocal calibration tasks)
* **primary key**:`SourceId` 
* **rows**: Contain sky sources. Be aware of flags `sky_source` and `detect_isPrimary`

*Questions:*
* We have no plan to match these to ObjectId for DP0.2. OK?
* 

*TODO:*
* Make all columns start lowercase.
* 
 
Example:

In [1]:
import lsst.daf.butler as dafButler
import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
butler = dafButler.Butler('/repo/dc2', collections=['2.2i/runs/test-med-1/w_2021_36/DM-31699'])
df = butler.get('sourceTable_visit', visit=212071)
print("The Source table is %s rows and %s columns" % (df.shape))
df.tail()

The Source table is 213182 rows and 152 columns


,sourceId,coord_ra,coord_dec,ccdVisitId,parentSourceId,x,y,xErr,yErr,ra,decl,ApFlux,ApFluxErr,sky,skyErr,PsFlux,PsFluxErr,Ixx,Iyy,Ixy,IxxPsf,IyyPsf,IxyPsf,extendedness,LocalPhotoCalib,LocalPhotoCalib_flag,LocalPhotoCalibErr,LocalWcs_flag,LocalWcs_CDMatrix_2_1,LocalWcs_CDMatrix_1_1,LocalWcs_CDMatrix_1_2,LocalWcs_CDMatrix_2_2,Blendedness_abs,Blendedness_flag,Blendedness_flag_noCentroid,Blendedness_flag_noShape,ApFlux_12_0_flag,ApFlux_12_0_flag_apertureTruncated,ApFlux_12_0_flag_sincCoeffsTruncated,ApFlux_12_0_instFlux,ApFlux_12_0_instFluxErr,ApFlux_17_0_flag,ApFlux_17_0_flag_apertureTruncated,ApFlux_17_0_instFlux,ApFlux_17_0_instFluxErr,ApFlux_25_0_flag,ApFlux_25_0_flag_apertureTruncated,ApFlux_25_0_instFlux,ApFlux_25_0_instFluxErr,ApFlux_35_0_flag,ApFlux_35_0_flag_apertureTruncated,ApFlux_35_0_instFlux,ApFlux_35_0_instFluxErr,ApFlux_3_0_flag,ApFlux_3_0_flag_apertureTruncated,ApFlux_3_0_flag_sincCoeffsTruncated,ApFlux_3_0_instFlux,ApFlux_3_0_instFluxErr,ApFlux_4_5_flag,ApFlux_4_5_flag_apertureTruncated,ApFlux_4_5_flag_sincCoeffsTruncated,ApFlux_4_5_instFlux,ApFlux_4_5_instFluxErr,ApFlux_50_0_flag,ApFlux_50_0_flag_apertureTruncated,ApFlux_50_0_instFlux,ApFlux_50_0_instFluxErr,ApFlux_6_0_flag,ApFlux_6_0_flag_apertureTruncated,ApFlux_6_0_flag_sincCoeffsTruncated,ApFlux_6_0_instFlux,ApFlux_6_0_instFluxErr,ApFlux_70_0_flag,ApFlux_70_0_flag_apertureTruncated,ApFlux_70_0_instFlux,ApFlux_70_0_instFluxErr,ApFlux_9_0_flag,ApFlux_9_0_flag_apertureTruncated,ApFlux_9_0_flag_sincCoeffsTruncated,ApFlux_9_0_instFlux,ApFlux_9_0_instFluxErr,Extendedness_flag,FootprintArea_value,Jacobian_flag,Jacobian_value,LocalBackground_instFlux,LocalBackground_instFluxErr,LocalBackground_flag,LocalBackground_flag_noGoodPixels,LocalBackground_flag_noPsf,PixelFlags,PixelFlags_bad,PixelFlags_cr,PixelFlags_crCenter,PixelFlags_edge,PixelFlags_interpolated,PixelFlags_interpolatedCenter,PixelFlags_offimage,PixelFlags_saturated,PixelFlags_saturatedCenter,PixelFlags_suspect,PixelFlags_suspectCenter,PsfFlux_apCorr,PsfFlux_apCorrErr,PsfFlux_area,PsfFlux_flag,PsfFlux_flag_apCorr,PsfFlux_flag_edge,PsfFlux_flag_noGoodPixels,Centroid_flag,Centroid_flag_almostNoSecondDerivative,Centroid_flag_badError,Centroid_flag_edge,Centroid_flag_noSecondDerivative,Centroid_flag_notAtMaximum,Centroid_flag_resetToPeak,Variance_flag,Variance_flag_emptyFootprint,Variance_value,Calib_astrometry_used,Calib_detected,Calib_photometry_reserved,Calib_photometry_used,Calib_psf_candidate,Calib_psf_reserved,Calib_psf_used,Deblend_deblendedAsPsf,Deblend_hasStrayFlux,Deblend_masked,Deblend_nChild,Deblend_parentTooBig,Deblend_patchedTemplate,Deblend_rampedTemplate,Deblend_skipped,Deblend_tooManyPeaks,HsmPsfMoments_flag,HsmPsfMoments_flag_no_pixels,HsmPsfMoments_flag_not_contained,HsmPsfMoments_flag_parent_source,HsmShapeRegauss_flag,HsmShapeRegauss_flag_galsim,HsmShapeRegauss_flag_no_pixels,HsmShapeRegauss_flag_not_contained,HsmShapeRegauss_flag_parent_source,sky_source,detect_isPrimary,band,instrument,detector,physical_filter,visit_system,visit
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
113854845131163562,113854845131163562,59.586569,-35.664513,212071175,113854845131163170,338.379693,3878.645317,1.592987,1.011278,59.586569,-35.664513,2747.277916,1044.830936,1.659222,49.050772,1572.264435,327.972988,10.658975,3.525215,0.940453,2.845071,2.895513,-0.076384,1.0,0.763623,False,0.0,False,8.428151e-07,-4.787249e-07,8.425011e-07,4.785934e-07,0.167913,False,False,False,False,False,False,3597.690918,1368.255737,False,False,4187.615466,1956.389364,False,False,7812.704708,2870.876832,False,False,8061.068128,4016.119098,False,False,False,1502.660767,340.319946,False,False,False,2093.156250,508.021454,False,False,11643.502559,5731.424402,False,False,False,2890.326660,680.833496,False,False,6665.067330,8020.572474,False,False,False,3610.148682,1024.866211,False,211,False,0.998947,2.172830,64.234316,False,Fal

##  The Object Table

* **dimensions:** tract 
* **specification:** https://github.com/lsst/obs_lsst/blob/master/policy/imsim/Object.yaml
* **what**: The product of multi-band forced photometry (fixed centroid) and measurement. All 6 bands included, contains colors and shapes. 
* **primary key**: `objectId` 
* **rows**: Contain sky objects and patch overlaps. Be aware of flags `sky_source` and `detect_isPrimary`


This table needs the most clean up. 

2 years ago we reconciled the DPDD with with what Science Pipelines has and wants to include. This reconciliation resulted in a list of tickets (TODO in the Object.yaml): https://jira.lsstcorp.org/browse/DM-22078 Please help. 

*Questions*
* How to do we want to advise users to get colors? for Galaxies? For Stars? Can I remove `grStd`, `riStd`, `izStd`, `zyStd` too?
* Does DC2 have Galactic extinction?
* 

*TODO*
* Lauren listed columns needed by the ol' pipe_analysis here: https://jira.lsstcorp.org/browse/DM-31339  (includes  'merge_peak_sky' among others). If we have columns that are taken from BOTH the `meas` and `forced` how do we disambiguate them by name?
* 

*Example:*

In [3]:
df = butler.get('objectTable', tract=3828, patch=5, skymap='DC2')
print("%s rows and %s columns" % (df.shape))
df.tail()

28948 rows and 857 columns


column,detect_isPatchInner,coord_ra,detect_isIsolated,refBand,zyStd,izStd,y,riStd,patchId,tractId,xErr,refFwhm,objectId,yErr,refExtendedness,detect_isPrimary,merge_peak_sky,grStd,parentObjectId,detect_isDeblendedSource,xy_flag,detect_isDeblendedModelSource,detect_fromBlend,coord_dec,detect_isTractInner,x,skymap,tract,patch,gRa,gDecl,gPsFlux,gPsFluxErr,gBdE1,gBdE2,gBdReB,gBdReD,gBdChi2,gBdFluxB,gBdFluxBErr,gBdFluxD,gBdFluxDErr,gGaapFlux_Psf,gGaapFluxErr_Psf,gGaapFlux_0_7,gGaapFluxErr_0_7,gGaapFlux_1_0,gGaapFluxErr_1_0,gGaapFlux_Optimal,gGaapFluxErr_Optimal,gIxx,gIyy,gIxy,gIxxPsf,gIyyPsf,gIxyPsf,gKronRad,gKronFlux,gKronFluxErr,gApFlux,gApFluxErr,gAp25Flux,gAp25FluxErr,gAp9Flux,gAp9FluxErr,gExtendedness,gBlendedness,gFwhm,gCModelFlux,gCModelFluxErr,gHsmShapeRegauss_e1,gHsmShapeRegauss_e2,gHsmShapeRegauss_flag,gInputCount,gCalib_psf_used,gPsfFlux_apCorr,gPsfFlux_apCorrErr,gPsfFlux_area,gPsfFlux_flag,gPsfFlux_flag_apCorr,gPsfFlux_flag_edge,gPsfFlux_flag_noGoodPixels,gGaapFlux_Psf_apCorr,gGaapFlux_Psf_apCorrErr,gGaapFlux_Optimal_apCorr,gGaapFlux_Optimal_apCorrErr,gGaapFlux_0_7_apCorr,gGaapFlux_0_7_apCorrErr,gGaapFlux_1_0_apCorr,gGaapFlux_1_0_apCorrErr,gGaapFlux_flag,gGaapFlux_flag_edge,gGaapFlux_flag_gaussianization,gGaapFlux_Optimal_flag_bigPsf,gGaapFlux_0_7_flag_bigPsf,gGaapFlux_1_0_flag_bigPsf,gKronFlux_apCorr,gKronFlux_apCorrErr,gKronFlux_flag,gKronFlux_flag_apCorr,gKronFlux_flag_bad_radius,gKronFlux_flag_bad_shape,gKronFlux_flag_bad_shape_no_psf,gKronFlux_flag_edge,gKronFlux_flag_no_fallback_radius,gKronFlux_flag_no_minimum_radius,gKronFlux_flag_small_radius,gKronFlux_flag_used_minimum_radius,gKronFlux_flag_used_psf_radius,gDeblend_nChild,gDeblend_skipped,gBlendedness_flag,gPixelFlags,gPixelFlags_bad,gPixelFlags_clipped,gPixelFlags_clippedCenter,gPixelFlags_cr,gPixelFlags_crCenter,gPixelFlags_edge,gPixelFlags_inexact_psf,gPixelFlags_inexact_psfCenter,gPixelFlags_interpolated,gPixelFlags_interpolatedCenter,gPixelFlags_offimage,gPixelFlags_saturated,gPixelFlags_saturatedCenter,gPixelFlags_sensor_edge,gPixelFlags_sensor_edgeCenter,gPixelFlags_suspect,gPixelFlags_suspectCenter,gExtendedness_flag,gInputCount_flag,gInputCount_flag_noInputs,gCalib_astrometry_used,gCalib_photometry_reserved,gCalib_photometry_used,gCalib_psf_candidate,gCalib_psf_reserved,gApFlux_flag,gApFlux_flag_apertureTruncated,gApFlux_flag_sincCoeffsTruncated,gCalibFlux_flag,gCalibFlux_flag_apertureTruncated,gCalibFlux_flag_sincCoeffsTruncated,gCentroid_flag_almostNoSecondDerivative,gCentroid_flag_edge,gCentroid_flag_noSecondDerivative,gCentroid_flag_notAtMaximum,gCentroid_flag_resetToPeak,gShapeRound_flag,gShapeRound_flag_no_pixels,gShapeRound_flag_not_contained,gShapeRound_flag_parent_source,gShapeRound_Flux,gShapeRound_x,gShapeRound_xx,gShapeRound_xy,gShapeRound_y,gShapeRound_yy,gPsfShape_flag,gPsfShape_flag_no_pixels,gPsfShape_flag_not_contained,gPsfShape_flag_parent_source,gShape_flag,gShape_flag_no_pixels,gShape_flag_not_contained,gShape_flag_parent_source,yRa,yDecl,yPsFlux,yPsFluxErr,yBdE1,yBdE2,yBdReB,yBdReD,yBdChi2,yBdFluxB,yBdFluxBErr,yBdFluxD,yBdFluxDErr,yGaapFlux_Psf,yGaapFluxErr_Psf,yGaapFlux_0_7,yGaapFluxErr_0_7,yGaapFlux_1_0,yGaapFluxErr_1_0,yGaapFlux_Optimal,yGaapFluxErr_Optimal,yIxx,yIyy,yIxy,yIxxPsf,yIyyPsf,yIxyPsf,yKronRad,yKronFlux,yKronFluxErr,yApFlux,yApFluxErr,yAp25Flux,yAp25FluxErr,yAp9Flux,yAp9FluxErr,yExtendedness,yBlendedness,yFwhm,yCModelFlux,yCModelFluxErr,yHsmShapeRegauss_e1,yHsmShapeRegauss_e2,yHsmShapeRegauss_flag,yInputCount,yCalib_psf_used,yPsfFlux_apCorr,yPsfFlux_apCorrErr,yPsfFlux_area,yPsfFlux_flag,yPsfFlux_flag_apCorr,yPsfFlux_flag_edge,yPsfFlux_flag_noGoodPixels,yGaapFlux_Psf_apCorr,yGaapFlux_Psf_apCorrErr,yGaapFlux_Optimal_apCorr,yGaapFlux_Optimal_apCorrErr,yGaapFlux_0_7_apCorr,yGaapFlux_0_7_apCorrErr,yGaapFlux_1_0_apCorr,yGaapFlux_1_0_apCorrErr,yGaapFlux_flag,yGaapFlux_flag_edge,yGaapFlux_flag_gaussianization,yGaapFlux_Optimal_flag_bigPsf,yGaapFlux_0_7_flag_bigPsf,yGaapFlux_1_0_flag_bigPsf,yKronFlux_apCorr,yKronFlux_apCorrErr

In [4]:
# The columns are sorted weird so there they are sorted
print('\n'.join(sorted(butler.get('objectTable.columns', tract=3828, patch=5, skymap='DC2'))))

coord_dec
coord_ra
detect_fromBlend
detect_isDeblendedModelSource
detect_isDeblendedSource
detect_isIsolated
detect_isPatchInner
detect_isPrimary
detect_isTractInner
gAp25Flux
gAp25FluxErr
gAp9Flux
gAp9FluxErr
gApFlux
gApFluxErr
gApFlux_flag
gApFlux_flag_apertureTruncated
gApFlux_flag_sincCoeffsTruncated
gBdChi2
gBdE1
gBdE2
gBdFluxB
gBdFluxBErr
gBdFluxD
gBdFluxDErr
gBdReB
gBdReD
gBlendedness
gBlendedness_flag
gCModelFlux
gCModelFluxErr
gCalibFlux_flag
gCalibFlux_flag_apertureTruncated
gCalibFlux_flag_sincCoeffsTruncated
gCalib_astrometry_used
gCalib_photometry_reserved
gCalib_photometry_used
gCalib_psf_candidate
gCalib_psf_reserved
gCalib_psf_used
gCentroid_flag_almostNoSecondDerivative
gCentroid_flag_edge
gCentroid_flag_noSecondDerivative
gCentroid_flag_notAtMaximum
gCentroid_flag_resetToPeak
gDeblend_nChild
gDeblend_skipped
gDecl
gExtendedness
gExtendedness_flag
gFwhm
gGaapFluxErr_0_7
gGaapFluxErr_1_0
gGaapFluxErr_Optimal
gGaapFluxErr_Psf
gGaapFlux_0_7
gGaapFlux_0_7_apCorr
gGaapFlux_

## The Visit and CcdVisit Table

### Visit

* **dimensions:** None, it's global. 
* **specification:**  https://github.com/lsst/pipe_tasks/blob/master/python/lsst/pipe/tasks/postprocess.py#L1375
* **what**:  Visits that went into the survey
* **primary key**: `visitId`

*Questions:*
* 


*TODO:* 

* Should prob make the DataFrame's index `visitId` and `ccdVisitId`
* `filterName` elsewhere has been populated with band. Is that what we want everywhere?
* 


In [5]:
df = butler.get('visitTable')
print("%s rows and %s columns" % (df.shape))
df.tail()

345 rows and 11 columns


,visitId,filterName,ra,decl,skyRotation,azimuth,altitude,zenithDistance,airmass,obsStart,expTime
340,491550,i_sim_1.4,53.449907,-35.929095,266.560170,249.394516,54.721704,35.278296,1.207819,2023-12-23 04:58:38.947099,30.0
341,492028,y_sim_1.4,54.238839,-38.354712,24.253633,207.413818,80.740185,9.259815,1.012015,2023-12-24 02:32:24.768000,30.0
342,496959,i_sim_1.4,55.103844,-34.622063,129.889181,250.214962,47.157621,42.842379,1.333820,2023-12-31 05:10:17.836900,30.0
343,496960,i_sim_1.4,55.433052,-36.679440,127.506435,247.232767,47.572124,42.427876,1.325870,2023-12-31 05:10:56.544000,30.0
344,496989,i_sim_1.4,56.084552,-35.792788,127.588507,247.831759,44.203987,45.796013,1.396866,2023-12-31 05:29:44.668900,30.0


### CcdVisit

* **dimensions:** None, it's global. 
* **specification:**  https://github.com/lsst/pipe_tasks/blob/master/python/lsst/pipe/tasks/postprocess.py#L1375
* **what**: 
* **primary key**: `ccdVisitId`

In [6]:
ccdVisit = butler.get('CcdVisitTable')
ccdVisit.tail()

,visitId,filterName,ra,decl,zenithDistance,zeroPoint,psfSigma,skyBg,skyNoise,ccdVisitId,seeing,skyRotation,expMidpt,expTime,obsStart,darkTime,xSize,ySize,llcra,llcdec,ulcra,ulcdec,urcra,urcdec,lrcra,lrcdec
id,,,,,,,,,,,,,,,,,,,,,,,,,,
184,496989,i_sim_1.4,58.001046,-36.075909,44.096111,31.845556,1.792896,3718.577393,72.746887,496989184,0.843047,127.588507,2023-12-31 05:29:44.668900,30.0,2023-12-31 05:29:29.668900,30.0,4071,3999,57.807700,-36.101017,57.973433,-35.919076,58.194168,-36.050500,58.028744,-36.232713
185,496989,i_sim_1.4,58.172089,-35.886951,43.986229,31.845991,1.803988,3718.831055,72.826355,496989185,0.848021,127.588507,2023-12-31 05:29:44.668900,30.0,2023-12-31 05:29:29.668900,30.0,4071,3999,57.979355,-35.912324,58.144152,-35.730175,58.364583,-35.861280,58.200105,-36.043707
186,496989,i_sim_1.4,58.062081,-36.403608,44.001713,31.847563,1.795575,3718.587646,72.831764,496989186,0.844026,127.588507,2023-12-31 05:29:44.668900,30.0,2023-12-31 05:29:29.668900,30.0,4071,3999,57.867992,-36.428906,58.034253,-36.246851,58.255937,-36.377998,58.089988,-36.560314
187,496989,i_sim_1.4,58.234409,-36.214566,43.890072,31.848886,1.793574,3719.165039,72.762138,496989187,0.842914,127.588507,2023-12-31 05:29:44.668900,30.0,2023-12-31 05:29:29.668900,30.0,4071,3999,58.040919,-36.240042,58.206373,-36.057841,58.427656,-36.188785,58.262523,-36.371253
188,496989,i_sim_1.4,58.405561,-36.025268,43.779480,31.848005,1.795980,3719.701416,72.788322,496989188,0.843763,127.588507,2023-12-31 05:29:44.668900,30.0,2023-12-31 05:29:29.668900,30.0,4071,3999,58.212698,-36.051028,58.377173,-35.868595,58.598163,-35.999206,58.434025,-36.181903


#  DIA Tables

$\color{red}{\text{Question}}$: The DIASourceTables pack flags into one column. Object, Source, ForcedSource tables that have boolean columns. Should we be consistent in the Parquet outputs from DRP in DP0.2?


## The Raw DIASource Table
* **dimensions**: `visit`
* **Specification:** https://github.com/lsst/ap_association/blob/master/data/DiaSource.yaml
* **what:** `diaSourceTable` is just an aggregated `goodSeeingDiff_diaSrcTable`. It has not yet been associated into Objects. Only purpose is QA. See next.

Q: Let's not bother ingesting this. 



In [7]:
df = butler.get('diaSourceTable', visit=212071)
print("The table is %s rows and %s columns" % (df.shape))
df.tail()

The table is 6629 rows and 36 columns


,diaSourceId,ccdVisitId,filterName,diaObjectId,parentDiaSourceId,midPointTai,pixelId,bboxSize,flags,ra,decl,x,y,xErr,yErr,apFlux,apFluxErr,snr,psFlux,psFluxErr,dipMeanFlux,dipMeanFluxErr,dipFluxDiff,dipFluxDiffErr,dipLength,dipAngle,dipChi2,isDipole,totFlux,totFluxErr,ixx,iyy,ixy,ixxPSF,iyyPSF,ixyPSF
76,113854797886521557,212071087,r,0,0,59867.214924,0,20,25165824,58.845664,-37.006881,1534.840074,3789.459778,NaN,NaN,131.512684,716.783426,0.183476,-49.576254,345.578937,4698.839573,NaN,81.057293,NaN,0.027158,-11.517111,2353.738494,True,68449.559944,331.678988,NaN,NaN,NaN,0.110835,-0.001505,-0.001505
77,113854797886521558,212071087,r,0,0,59867.214924,0,15,0,58.865078,-37.034306,967.430847,3791.172119,NaN,NaN,1360.589436,679.561534,2.002158,1096.246765,206.938536,1584.698964,NaN,1074.240524,NaN,NaN,NaN,NaN,False,996.665020,203.484197,0.066791,0.067258,-0.024602,0.110820,-0.001505,-0.001505
78,113854797886521559,212071087,r,0,0,59867.214924,0,14,0,58.883882,-37.051621,563.073120,3874.134033,NaN,NaN,-777.598075,675.877665,-1.150501,1119.937813,208.262442,1233.552889,NaN,649.370080,NaN,NaN,NaN,NaN,False,1141.266538,204.055555,0.104554,0.074490,-0.016670,0.110809,-0.001505,-0.001505
79,113854797886521560,212071087,r,0,0,59867.214924,0,19,25165824,58.864391,-37.021208,1177.960658,3897.962237,NaN,NaN,505.765540,686.469884,0.736763,-235.646828,268.511643,2688.509379,NaN,112.457213,NaN,0.040057,-18.615914,2329.710710,True,26909.450563,259.175180,0.003331,0.003331,0.000000,0.110825,-0.001505,-0.001505
80,113854797886521561,212071087,r,0,0,59867.214924,0,23,25165824,58.839842,-36.979575,2004.641928,3956.943040,NaN,NaN,-3548.747000,953.385905,-3.722260,-2486.878569,781.782987,9214.587788,NaN,-3472.609729,NaN,0.008682,-133.796856,8247.339609,True,509935.669325,758.067813,NaN,NaN,NaN,0.110847,-0.001505,-0.001505


## The DIASource Table

* **dimensions:** tract, patch
* **specification:** https://github.com/lsst/ap_association/blob/master/data/DiaSource.yaml
* **what**: DiaSource Table with diaObjectId for matching to the diaObject Table. 
* **primary key**: `diaSourceId`, foreign keys: (`diaObjectId`, `ccdVisitId`)

Question: Should we aggregate this into a bigger per-tract table? 

In [8]:
df = butler.get('goodSeeingDiff_assocDiaSrcTable', tract=3828, patch=3)
print("The table is %s rows and %s columns" % (df.shape))
df.tail()

The table is 5035 rows and 36 columns


,ccdVisitId,diaSourceId,filterName,diaObjectId,parentDiaSourceId,midPointTai,pixelId,bboxSize,flags,ra,decl,x,y,xErr,yErr,apFlux,apFluxErr,snr,psFlux,psFluxErr,dipMeanFlux,dipMeanFluxErr,dipFluxDiff,dipFluxDiffErr,dipLength,dipAngle,dipChi2,isDipole,totFlux,totFluxErr,ixx,iyy,ixy,ixxPSF,iyyPSF,ixyPSF
5030,183772019,98661851440611484,g,3299854297281331488,0,59823.289126,0,39,25296896,56.601205,-37.075319,20.609337,1929.650269,None,None,NaN,NaN,NaN,NaN,NaN,2028.549295,NaN,2252.508387,NaN,NaN,NaN,NaN,False,930.324334,112.728001,NaN,NaN,NaN,0.243043,0.002566,0.002566
5031,183772019,98661851440611490,g,3299854297281331250,0,59823.289126,0,50,25165824,56.520353,-37.067885,1118.337152,2332.342612,None,None,681.572084,531.572514,1.282181,-4473.353070,579.680867,9457.864009,NaN,2981.951716,NaN,0.012755,-172.065993,1490.119660,True,301622.332512,540.647869,NaN,NaN,NaN,0.243036,0.002565,0.002565
5032,183772019,98661851440611494,g,3299854297281331427,0,59823.289126,0,19,25165824,56.572759,-37.041657,275.099487,2615.986328,None,None,-56.902023,276.075083,-0.206111,-827.541583,166.343832,1149.238586,NaN,-231.847228,NaN,NaN,NaN,NaN,False,14228.197418,157.645654,NaN,NaN,NaN,0.243081,0.002566,0.002566
5033,183772019,98661851440611495,g,3299854297281331227,0,59823.289126,0,54,25165824,56.554277,-37.036235,510.348794,2773.352172,None,None,-1504.073365,580.622432,-2.590450,-13597.210369,624.756783,15820.001150,NaN,3290.369566,NaN,0.448537,1.796040,254775.958747,True,358331.820575,583.893961,NaN,NaN,NaN,0.243084,0.002566,0.002566
5034,183772019,98661851440611496,g,3299854297281331229,0,59823.289126,0,36,25165824,56.576823,-37.031881,176.876216,2773.737787,None,None,156.346117,416.745821,0.375159,-1689.587923,418.871827,5381.885902,NaN,686.560428,NaN,0.021333,162.958117,728.219295,True,157833.128688,398.369401,NaN,NaN,NaN,0.243095,0.002566,0.002566


## The DIAObject Table

* **dimensions:** tract, patch
* **specification:** https://github.com/lsst/meas_base/blob/3bd782b489b14a3f46934e5e0dd43256f727c0fe/python/lsst/meas/base/diaCalculationPlugins.py
* **what**:  `DiaObjects` with aggregate metrics computes on associated `DiaSources`.
* **primary key**: `diaObjectId`, foreign keys: (`diaObjectId`, `ccdVisitId`)

Questions:
* Are the DiaSources filtered on flags before matching? what 
* What packer is used for the flag columns

In [9]:
df = butler.get('goodSeeingDiff_fullDiaObjTable', tract=3828, patch=3)
print("The table is %s rows and %s columns" % (df.shape))
df.tail()

The table is 1377 rows and 139 columns


,diaObjectId,ra,decl,nDiaSources,flags,radecTai,pixelId,uPSFluxLinearSlope,uPSFluxLinearIntercept,uPSFluxMAD,uPSFluxMaxSlope,uPSFluxErrMean,uPSFluxMean,uPSFluxMeanErr,uPSFluxNdata,uTOTFluxMean,uTOTFluxMeanErr,uPSFluxMin,uPSFluxMax,uPSFluxPercentile05,uPSFluxPercentile25,uPSFluxPercentile50,uPSFluxPercentile75,uPSFluxPercentile95,uPSFluxSigma,uTOTFluxSigma,uPSFluxSkew,uPSFluxChi2,uPSFluxStetsonJ,gPSFluxLinearSlope,gPSFluxLinearIntercept,gPSFluxMAD,gPSFluxMaxSlope,gPSFluxErrMean,gPSFluxMean,gPSFluxMeanErr,gPSFluxNdata,gTOTFluxMean,gTOTFluxMeanErr,gPSFluxMin,gPSFluxMax,gPSFluxPercentile05,gPSFluxPercentile25,gPSFluxPercentile50,gPSFluxPercentile75,gPSFluxPercentile95,gPSFluxSigma,gTOTFluxSigma,gPSFluxSkew,gPSFluxChi2,gPSFluxStetsonJ,rPSFluxLinearSlope,rPSFluxLinearIntercept,rPSFluxMAD,rPSFluxMaxSlope,rPSFluxErrMean,rPSFluxMean,rPSFluxMeanErr,rPSFluxNdata,rTOTFluxMean,rTOTFluxMeanErr,rPSFluxMin,rPSFluxMax,rPSFluxPercentile05,rPSFluxPercentile25,rPSFluxPercentile50,rPSFluxPercentile75,rPSFluxPercentile95,rPSFluxSigma,rTOTFluxSigma,rPSFluxSkew,rPSFluxChi2,rPSFluxStetsonJ,iPSFluxLinearSlope,iPSFluxLinearIntercept,iPSFluxMAD,iPSFluxMaxSlope,iPSFluxErrMean,iPSFluxMean,iPSFluxMeanErr,iPSFluxNdata,iTOTFluxMean,iTOTFluxMeanErr,iPSFluxMin,iPSFluxMax,iPSFluxPercentile05,iPSFluxPercentile25,iPSFluxPercentile50,iPSFluxPercentile75,iPSFluxPercentile95,iPSFluxSigma,iTOTFluxSigma,iPSFluxSkew,iPSFluxChi2,iPSFluxStetsonJ,zPSFluxLinearSlope,zPSFluxLinearIntercept,zPSFluxMAD,zPSFluxMaxSlope,zPSFluxErrMean,zPSFluxMean,zPSFluxMeanErr,zPSFluxNdata,zTOTFluxMean,zTOTFluxMeanErr,zPSFluxMin,zPSFluxMax,zPSFluxPercentile05,zPSFluxPercentile25,zPSFluxPercentile50,zPSFluxPercentile75,zPSFluxPercentile95,zPSFluxSigma,zTOTFluxSigma,zPSFluxSkew,zPSFluxChi2,zPSFluxStetsonJ,yPSFluxLinearSlope,yPSFluxLinearIntercept,yPSFluxMAD,yPSFluxMaxSlope,yPSFluxErrMean,yPSFluxMean,yPSFluxMeanErr,yPSFluxNdata,yTOTFluxMean,yTOTFluxMeanErr,yPSFluxMin,yPSFluxMax,yPSFluxPercentile05,yPSFluxPercentile25,yPSFluxPercentile50,yPSFluxPercentile75,yPSFluxPercentile95,yPSFluxSigma,yTOTFluxSigma,yPSFluxSkew,yPSFluxChi2,yPSFluxStetsonJ
diaObjectId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3299854297281332573,3299854297281332573,56.518281,-37.042134,1,1,60309.228989,9.870792e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,355.557972,5052.123697,355.557972,1.0,5045.649927,346.930975,5052.123697,5052.123697,5052.123697,5052.123697,5052.123697,5052.123697,5052.123697,NaN,NaN,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3299854297281332574,3299854297281332574,56.616917,-37.167577,1,1,60309.228989,9.870794e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,334.111282,-1690.705781,334.111282,1.0,-1737.581564,333.015473,-1690.705781,-1690.705781,-1690.705781,-1690.705781,-1690.705781,-1690.705781,-1690.705781,NaN,NaN,NaN,4.631248e-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3299854297281332575,3299854297281332575,56.662829,-37.221311,1,1,60309.228989,9.870806e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [10]:
butler = dafButler.Butler('/repo/dc2', collections=['u/yusra/DC2/old-bps/DM-28394'])

# ForcedSource Table

The plan is to associate the `DiaObjects` and `CoaddSources` (what we are calling objects in the pipelines today) into final `Objects` before performing forced photometry on both the difference images and calexps. For DP0.2, we're proposing to skip this association step, so that users can choose whether they want to look at lightcurves of the sources detected on coadds, or the sources detected on difference images. 
* Pro: better for QA. Easier to answer questions like "where did this junk detection come from?" and to keep our very new DRP DC2 DIA pipeline separate from the more battle tested one. 
* Con: Users are going to think it's always going to be this way, and may get used to it. 

## ForcedSource

* **dimensions:** tract
* **specification:** https://github.com/lsst/obs_lsst/blob/master/pipelines/imsim/DRP.yaml
* **what**: Object centroids (from `deepCoadd_ref`) forced photometered on calexps and diffims in one table.  
* **primary key**: non existent currently. foreign keys: (`objectId`, `ccdVisitId`)

Questions:
* `band` or `filterName` on all?
* Includes patch overlaps on purpose so that users can use the overlaps for QA or remove using `detect_isPrimary` or `detect_isPatchInner` for analysis. OK?

TODO:
* appears to be missing a primary key.
* flags are all False. 
* Add local calib columns


In [11]:
butler = dafButler.Butler('/repo/dc2', collections=['u/yusra/DC2/old-bps/DM-28394'])
df = butler.get('ForcedSourceTable_tract', tract=3828)
print("The table is %s rows and %s columns" % (df.shape))
df.tail()

The table is 4731729 rows and 35 columns


,detect_isPrimary,detect_isTractInner,detect_isPatchInner,objectId,parentObjectId,coord_ra,coord_dec,ccdVisitId,band,PsFlux,PsFluxErr,PsFluxFlag,psFluxApCorr,psFluxApCorrErr,PsDiffFlux,PsDiffFluxErr,PsDiffFluxFlag,psDiffFluxApCorr,psDiffFluxApCorrErr,localBackground_instFlux,localBackground_instFluxErr,pixelFlags,pixelFlags_edge,pixelFlags_interpolated,pixelFlags_saturated,pixelFlags_cr,pixelFlags_bad,pixelFlags_suspect,pixelFlags_interpolatedCenter,pixelFlags_saturatedCenter,pixelFlags_crCenter,pixelFlags_suspectCenter,skymap,tract,patch
1649962333012797647,False,True,False,1649962333012797647,0,57.343880,-36.855717,228020115,i,76.166886,259.896934,False,0.979562,0.0,-71.921595,248.190933,False,0.979562,0.0,-1.525026,68.478883,False,False,False,False,False,False,False,False,False,False,False,DC2,3828,7
1649962333012797649,False,True,True,1649962333012797649,0,57.351256,-36.849351,228020115,i,269.240720,260.213646,False,0.978902,0.0,265.512093,248.371402,False,0.978902,0.0,0.612889,71.841647,False,False,False,False,False,False,False,False,False,False,False,DC2,3828,7
1649962333012797650,False,True,False,1649962333012797650,0,57.342839,-36.849067,228020115,i,-127.663248,258.736629,False,0.980635,0.0,-65.135659,247.298520,False,0.980635,0.0,-1.243882,70.214145,False,False,False,False,False,False,False,False,False,False,False,DC2,3828,7
1649962333012797653,False,True,True,1649962333012797653,0,57.362364,-36.840114,228020115,i,427.288634,259.833468,True,0.977889,0.0,NaN,NaN,True,0.977889,0.0,2.321080,67.406001,False,False,False,False,False,False,False,False,False,False,False,DC2,3828,7
1649962333012797655,False,True,True,1649962333012797655,0,57.349442,-36.833464,228020115,i,-326.638206,258.624514,False,0.981509,0.0,-312.112720,245.698169,False,0.981509,0.0,-1.556618,69.416625,False,False,False,False,False,False,False,False,False,False,False,DC2,3828,7


## ForcedSourceOnDiaObjects Table

Currently on branch DM-31765. 

Section to be added in the morning. 
